# Spark Structured Streaming: Complete Guide

---

### Slide 1: Title Slide
# Apache Spark Structured Streaming
## A Comprehensive Guide to Stream Processing

**Subtitle:** From Basic Concepts to Advanced Implementation
**Version:** Apache Spark 3.5.1

---

### Slide 2: What is Structured Streaming?

## Definition
- **Scalable and fault-tolerant** stream processing engine
- Built on **Spark SQL engine**
- Express streaming computation same way as **batch computation**
- **Incremental and continuous** execution
- **End-to-end exactly-once** fault-tolerance guarantees

## Key Benefits
✅ **Fast** - Low latency processing (100ms+)  
✅ **Scalable** - Handles massive data volumes  
✅ **Fault-tolerant** - Automatic recovery from failures  
✅ **Unified** - Same API for batch and streaming  
✅ **Exactly-once** - No data loss or duplication  

---

### Slide 3: Processing Models

## Two Processing Modes

### 1. Micro-batch Processing (Default)
- Processes data as **small batch jobs**
- **End-to-end latencies**: 100 milliseconds+
- **Exactly-once** fault-tolerance guarantees
- Best for most use cases

### 2. Continuous Processing
- **Ultra-low latency**: 1 millisecond+
- **At-least-once** guarantees
- Available since Spark 2.3
- For latency-critical applications

```python
# Micro-batch (default)
query = df.writeStream.start()

# Continuous processing
query = df.writeStream.trigger(continuous='1 second').start()
```

---

### Slide 4: Programming Model - Core Concept

## The "Infinite Table" Model

### Input Stream = Unbounded Table
- Each data record = New row appended to table
- Streaming computation = Query on infinite table
- Results updated incrementally as new data arrives

### Visual Representation:
```
Input Stream     →     Input Table      →     Result Table     →     Output
[data1, data2]          | data1 |             | result1 |           | result1 |
[data3, data4]          | data2 |      →      | result2 |     →     | result2 |
[data5, data6]          | data3 |             | result3 |           | result3 |
                        | data4 |
                        | data5 |
                        | data6 |
```

---

### Slide 5: Quick Example - Streaming Word Count

## Python Code Example

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create SparkSession
spark = SparkSession.builder \
    .appName("StructuredStreamingWordCount") \
    .getOrCreate()

# Create streaming DataFrame from socket
lines = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Transform: split lines into words and count
words = lines.select(explode(split(lines.value, " ")).alias("word"))
wordCounts = words.groupBy("word").count()

# Output to console
query = wordCounts.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()
```

---

### Slide 6: Output Modes

## Three Output Modes

### 1. Append Mode (Default)
- Only **new rows** added to Result Table
- Each row output **only once**
- Best for: select, where, map, filter, joins

### 2. Complete Mode
- **Entire Result Table** output every trigger
- Best for: aggregation queries
- Higher resource usage

### 3. Update Mode
- Only **updated rows** since last trigger
- Available since Spark 2.1.1
- Balance between Append and Complete

```python
# Append mode (default)
query = df.writeStream.outputMode("append").start()

# Complete mode
query = df.writeStream.outputMode("complete").start()

# Update mode
query = df.writeStream.outputMode("update").start()
```

---

### Slide 7: Creating Streaming DataFrames

## Input Sources

### 1. File Source
```python
# Read streaming files
df = spark.readStream \
    .format("json") \
    .option("path", "/path/to/directory") \
    .option("maxFilesPerTrigger", 1) \
    .load()
```

### 2. Kafka Source
```python
# Read from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "topic1") \
    .load()
```

### 3. Socket Source (Testing)
```python
# Read from socket (testing only)
df = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()
```

---

### Slide 8: Source Compatibility Matrix

## Fault-Tolerance Support

| Source | Fault-Tolerant | Notes |
|--------|----------------|-------|
| **File Source** | ✅ Yes | Supports most file formats |
| **Kafka Source** | ✅ Yes | Production recommended |
| **Socket Source** | ❌ No | Testing only |
| **Rate Source** | ✅ Yes | Benchmarking/testing |

## File Source Options
```python
df = spark.readStream \
    .format("parquet") \
    .option("path", "/data/events/") \
    .option("maxFilesPerTrigger", 10) \
    .option("latestFirst", "true") \
    .option("fileNameOnly", "false") \
    .load()
```

---

### Slide 9: Basic Operations

## Supported Operations

### Selection, Projection, Aggregation
```python
# Selection and filtering
filtered_df = streaming_df.select("name", "age") \
    .where("age > 21")

# Aggregation
agg_df = streaming_df.groupBy("category") \
    .agg(count("*").alias("count"), 
         avg("price").alias("avg_price"))

# Register as temp view
streaming_df.createOrReplaceTempView("streaming_data")
result = spark.sql("SELECT category, COUNT(*) FROM streaming_data GROUP BY category")
```

### Check if DataFrame is Streaming
```python
if df.isStreaming:
    print("This is a streaming DataFrame")
else:
    print("This is a static DataFrame")
```

---

### Slide 10: Window Operations - Time-based Aggregations

## Event-Time Windows

### Concept
- Aggregate data based on **event timestamps**
- Handle **late-arriving data**
- **Tumbling** vs **Sliding** windows

### Python Example - Sliding Window
```python
from pyspark.sql.functions import window

# 10-minute windows, sliding every 5 minutes
windowed_counts = events \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(col("timestamp"), "10 minutes", "5 minutes"),
        col("word")
    ) \
    .count()

# Output schema: [window, word, count]
query = windowed_counts.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()
```

---

### Slide 11: Window Types Visual

## Three Types of Time Windows

### 1. Tumbling Windows (Fixed)
```
|----10min----|----10min----|----10min----|
12:00    12:10    12:20    12:30
```

### 2. Sliding Windows (Overlapping)
```
|----10min----|
     |----10min----|
          |----10min----|
12:00  12:05  12:10  12:15
```

### 3. Session Windows (Dynamic)
```python
# Session windows with gap duration
session_df = events.groupBy(
    session_window(col("timestamp"), "5 minutes"),
    col("user_id")
).count()

# Dynamic gap duration
session_df = events.groupBy(
    session_window(col("timestamp"), 
                  when(col("activity_type") == "click", "30 seconds")
                  .otherwise("5 minutes")),
    col("user_id")
).count()
```

---

### Slide 12: Watermarking - Handling Late Data

## What is Watermarking?

### Problem
- **Late data** can arrive out of order
- System needs to know when to **stop waiting** for late data
- **Memory management** for stateful operations

### Solution: Watermarks
- Define **maximum expected delay** for late data
- Automatically **clean up old state**
- Balance between **correctness** and **resource usage**

```python
# Watermark example
result = events \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(col("timestamp"), "5 minutes"),
        col("word")
    ) \
    .count()
```

---

### Slide 13: Watermarking Visual Example

## How Watermarking Works

### Timeline Example:
```
Current Time: 12:14
Watermark: 12:04 (10 minutes behind)

Data Processing:
- (12:07, "cat") → ✅ Processed (within watermark)
- (12:09, "dog") → ✅ Processed (within watermark)  
- (12:02, "bird") → ❌ Dropped (beyond watermark)
```

### Watermark Calculation
```
Watermark = Max Event Time Seen - Watermark Delay
```

### Python Implementation
```python
# Set watermark delay
df_with_watermark = streaming_df \
    .withWatermark("event_time", "10 minutes")

# Use in aggregation
result = df_with_watermark \
    .groupBy(window("event_time", "5 minutes")) \
    .count()
```

---

### Slide 14: Join Operations

## Types of Joins Supported

### 1. Stream-Static Joins
```python
# Join streaming data with static reference data
static_df = spark.read.table("user_profiles")
streaming_df = spark.readStream.table("user_events")

joined = streaming_df.join(static_df, "user_id")
```

### 2. Stream-Stream Joins
```python
# Join two streaming DataFrames
impressions = spark.readStream.table("ad_impressions")
clicks = spark.readStream.table("ad_clicks")

# Inner join with time constraints
result = impressions \
    .withWatermark("imp_time", "2 hours") \
    .join(
        clicks.withWatermark("click_time", "3 hours"),
        expr("""
            impression_id = click_impression_id AND
            click_time >= imp_time AND
            click_time <= imp_time + interval 1 hour
        """)
    )
```

---

### Slide 15: Join Support Matrix

## Supported Join Types

| Left Input | Right Input | Join Type | Support |
|------------|-------------|-----------|---------|
| **Static** | **Static** | All types | ✅ Always |
| **Stream** | **Static** | Inner, Left Outer, Semi | ✅ Stateless |
| **Static** | **Stream** | Inner, Right Outer | ✅ Stateless |
| **Stream** | **Stream** | Inner | ✅ With watermark |
| **Stream** | **Stream** | Outer | ✅ With watermark + constraints |

### Important Notes:
- Stream-stream joins **require watermarks** for state cleanup
- **Time constraints** needed for outer joins
- **Append mode only** for joins (as of Spark 2.4+)

---

### Slide 16: Streaming Deduplication

## Remove Duplicate Records

### With Watermark (Recommended)
```python
# Deduplicate with watermark
deduplicated = streaming_df \
    .withWatermark("timestamp", "1 hour") \
    .dropDuplicates(["user_id", "transaction_id"])

# Within watermark window
deduplicated_windowed = streaming_df \
    .withWatermark("timestamp", "1 hour") \
    .dropDuplicatesWithinWatermark(["user_id", "transaction_id"])
```

### Without Watermark (Unbounded State)
```python
# Caution: State grows indefinitely
deduplicated = streaming_df \
    .dropDuplicates(["user_id", "transaction_id"])
```

### Use Cases:
- **Exactly-once processing** from unreliable sources
- **Data quality** improvement
- **Idempotent operations**

---

### Slide 17: Arbitrary Stateful Operations

## Advanced State Management

### mapGroupsWithState
```python
from pyspark.sql.streaming import GroupState
from pyspark.sql.types import *

# Define state update function
def update_user_session(key, values, state):
    if state.hasTimedOut:
        return []
    
    # Update state logic
    current_state = state.getOption if state.exists else {}
    
    # Process values and update state
    for value in values:
        # Custom logic here
        pass
    
    state.update(new_state)
    state.setTimeoutDuration("10 minutes")
    
    return [output_row]

# Apply stateful operation
result = streaming_df \
    .groupByKey(lambda x: x.user_id) \
    .mapGroupsWithState(
        outputStructType,
        stateStructType,
        GroupStateTimeout.ProcessingTimeTimeout
    )(update_user_session)
```

---

### Slide 18: Output Sinks

## Built-in Output Sinks

### 1. File Sink
```python
query = df.writeStream \
    .format("parquet") \
    .option("path", "/output/path") \
    .option("checkpointLocation", "/checkpoint/path") \
    .partitionBy("year", "month") \
    .start()
```

### 2. Kafka Sink
```python
query = df.select(to_json(struct("*")).alias("value")) \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "output-topic") \
    .start()
```

### 3. Console Sink (Debugging)
```python
query = df.writeStream \
    .format("console") \
    .option("numRows", 20) \
    .option("truncate", "false") \
    .start()
```

---

### Slide 19: Sink Capabilities Matrix

## Sink Support and Features

| Sink Type | Output Modes | Fault-Tolerant | Use Case |
|-----------|-------------|----------------|----------|
| **File** | Append | ✅ Exactly-once | Data Lake, Analytics |
| **Kafka** | Append, Update, Complete | ✅ At-least-once | Message Streaming |
| **Console** | Append, Update, Complete | ❌ Debug only | Development |
| **Memory** | Append, Complete | ❌ Debug only | Testing |
| **ForeachBatch** | All modes | Depends on impl. | Custom Logic |

### Custom Sink Example:
```python
def process_batch(df, epoch_id):
    # Custom processing logic
    df.write.mode("append").saveAsTable("my_table")
    
query = streaming_df.writeStream \
    .foreachBatch(process_batch) \
    .start()
```

---

### Slide 20: Triggers - Controlling Execution

## Trigger Types

### 1. Default (Micro-batch)
```python
# Process as fast as possible
query = df.writeStream.start()
```

### 2. Fixed Interval
```python
# Process every 30 seconds
query = df.writeStream \
    .trigger(processingTime='30 seconds') \
    .start()
```

### 3. One-time (Batch)
```python
# Process available data once and stop
query = df.writeStream \
    .trigger(once=True) \
    .start()
```

### 4. Continuous
```python
# Ultra-low latency processing
query = df.writeStream \
    .trigger(continuous='1 second') \
    .start()
```

---

### Slide 21: Checkpointing and Fault Tolerance

## Checkpoint Location

### Purpose:
- **Recovery** from failures
- **Exactly-once** processing guarantees
- **State management** for stateful operations
- **Progress tracking**

### Implementation:
```python
query = df.writeStream \
    .option("checkpointLocation", "s3://bucket/checkpoints/query1") \
    .format("delta") \
    .outputMode("append") \
    .start()
```

### What's Stored:
- **Offsets** - Progress in input sources
- **State** - Intermediate computation state  
- **Metadata** - Query configuration
- **WAL** - Write-ahead logs

---

### Slide 22: Monitoring Streaming Queries

## Query Management and Monitoring

### Basic Monitoring:
```python
# Start query
query = df.writeStream \
    .queryName("my_streaming_query") \
    .format("console") \
    .start()

# Monitor progress
print(f"Query ID: {query.id}")
print(f"Query Name: {query.name}")
print(f"Is Active: {query.isActive}")

# Get progress information
progress = query.lastProgress
print(f"Batch ID: {progress['batchId']}")
print(f"Input Rows: {progress['inputRowsPerSecond']}")
print(f"Process Rows: {progress['processedRowsPerSecond']}")

# Wait for termination
query.awaitTermination()
```

### Advanced Monitoring:
```python
# Custom progress listener
class MyProgressListener:
    def onQueryProgress(self, event):
        progress = event.progress
        print(f"Batch {progress.batchId} processed {progress.numInputRows} rows")

spark.streams.addListener(MyProgressListener())
```

---

### Slide 23: State Store Management

## State Store Types

### 1. HDFS State Store (Default)
- **In-memory** state with **HDFS backup**
- **Versioned** key-value store
- Good for **moderate state sizes**

### 2. RocksDB State Store
```python
# Configure RocksDB state store
spark.conf.set(
    "spark.sql.streaming.stateStore.providerClass",
    "org.apache.spark.sql.execution.streaming.state.RocksDBStateStoreProvider"
)

# RocksDB configuration
spark.conf.set("spark.sql.streaming.stateStore.rocksdb.changelogCheckpointing.enabled", "true")
spark.conf.set("spark.sql.streaming.stateStore.rocksdb.compactOnCommit", "false")
```

### When to Use RocksDB:
- **Millions of keys** in state
- **Large state size** (> JVM memory)
- **GC pressure** issues
- **Better performance** for large state

---

### Slide 24: Performance Optimization

## Best Practices for Performance

### 1. Resource Configuration
```python
spark.conf.set("spark.sql.streaming.metricsEnabled", "true")
spark.conf.set("spark.sql.streaming.numRecentProgressUpdates", "10")
spark.conf.set("spark.sql.adaptive.enabled", "true")
```

### 2. Partitioning Strategy
```python
# Repartition for better parallelism
repartitioned = streaming_df.repartition(col("partition_key"))

# Write with partitioning
query = repartitioned.writeStream \
    .partitionBy("date", "hour") \
    .format("delta") \
    .start()
```

### 3. State Optimization
```python
# Use watermarks to limit state size
optimized = streaming_df \
    .withWatermark("timestamp", "1 hour") \
    .groupBy("key") \
    .count()
```

---

### Slide 25: Error Handling and Debugging

## Common Issues and Solutions

### 1. Schema Inference
```python
# Disable schema inference for files (recommended)
spark.conf.set("spark.sql.streaming.schemaInference", "false")

# Provide explicit schema
from pyspark.sql.types import *

schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("user_id", StringType(), True),
    StructField("action", StringType(), True)
])

df = spark.readStream \
    .schema(schema) \
    .format("json") \
    .load("/path/to/data")
```

### 2. Debugging Techniques
```python
# Debug with console sink
debug_query = streaming_df.writeStream \
    .format("console") \
    .option("numRows", 50) \
    .start()

# Check query plan
streaming_df.explain(True)

# Monitor metrics
print(query.status)
print(query.recentProgress)
```

---

### Slide 26: Complete Example - Real-time Analytics

## End-to-End Implementation

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create Spark session
spark = SparkSession.builder \
    .appName("RealTimeAnalytics") \
    .config("spark.sql.streaming.checkpointLocation.deleteOnExit", "true") \
    .getOrCreate()

# Define schema
schema = StructType([
    StructField("timestamp", TimestampType()),
    StructField("user_id", StringType()),
    StructField("product_id", StringType()),
    StructField("action", StringType()),
    StructField("price", DoubleType())
])

# Read from Kafka
raw_events = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "user-events") \
    .load()

# Parse JSON data
events = raw_events.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# Real-time aggregations with watermarking
hourly_stats = events \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window("timestamp", "1 hour", "15 minutes"),
        "action"
    ) \
    .agg(
        count("*").alias("event_count"),
        sum("price").alias("total_revenue"),
        countDistinct("user_id").alias("unique_users")
    )

# Write to Delta Lake
query = hourly_stats.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/checkpoint") \
    .table("real_time_analytics")

query.awaitTermination()
```

---

### Slide 27: Advanced Features Summary

## Key Advanced Capabilities

### ✅ **Watermarking**
- Handle late-arriving data
- Automatic state cleanup
- Configurable delay thresholds

### ✅ **Multiple Join Types**
- Stream-static joins
- Stream-stream joins with time constraints
- Outer joins with watermarks

### ✅ **Stateful Operations**
- Custom state management
- Session tracking
- Complex event processing

### ✅ **Fault Tolerance**
- Exactly-once processing
- Automatic recovery
- Checkpoint-based recovery

### ✅ **Performance Features**
- RocksDB state store
- Adaptive query execution
- Predicate pushdown

---

### Slide 28: Migration and Compatibility

## Upgrading and Migration

### From DStreams to Structured Streaming
```python
# Old DStreams approach (deprecated)
# ssc = StreamingContext(sc, 2)
# lines = ssc.socketTextStream("localhost", 9999)
# words = lines.flatMap(lambda line: line.split(" "))

# New Structured Streaming approach
lines = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

words = lines.select(explode(split("value", " ")).alias("word"))
```

### Version Compatibility
- **Checkpoint compatibility** across minor versions
- **Schema evolution** support
- **Graceful degradation** for unsupported features

---

### Slide 29: Production Deployment Checklist

## Production Readiness

### ✅ **Resource Planning**
- Memory sizing for state stores
- CPU cores for parallelism  
- Network bandwidth for throughput

### ✅ **Monitoring Setup**
- Query progress tracking
- Custom metrics collection
- Alerting on failures

### ✅ **Fault Tolerance**
- Reliable checkpoint storage
- Multiple Kafka brokers
- Cluster redundancy

### ✅ **Testing Strategy**
- Unit tests for transformations
- Integration tests with real data
- Load testing for capacity planning

### ✅ **Operational Procedures**
- Deployment automation
- Rollback procedures
- Monitoring dashboards

---

### Slide 30: Best Practices Summary

## Key Recommendations

### 🎯 **Schema Management**
- Always provide **explicit schemas**
- Plan for **schema evolution**
- Use **compatible data types**

### 🎯 **State Management**  
- Use **watermarks** appropriately
- Choose **right state store** (HDFS vs RocksDB)
- Monitor **state size growth**

### 🎯 **Performance**
- **Partition data** appropriately  
- **Tune trigger intervals**
- **Optimize join conditions**

### 🎯 **Reliability**
- Use **fault-tolerant sources/sinks**
- Configure **proper checkpointing**
- Implement **monitoring and alerting**

### 🎯 **Testing**
- Test with **realistic data volumes**
- Validate **exactly-once semantics**
- Plan for **failure scenarios**

---

### Slide 31: Resources and Next Steps

## Learning Resources

### 📖 **Official Documentation**
- Spark Structured Streaming Guide
- API Documentation (Python/Scala/Java)
- Configuration Reference

### 🛠 **Hands-on Practice**
- Databricks Community Edition
- Local Spark cluster setup
- Sample datasets and tutorials

### 🏢 **Production Examples**
- Netflix - Real-time recommendations
- Uber - Surge pricing algorithms  
- Airbnb - Fraud detection systems

### 🔗 **Community Resources**
- Spark user mailing lists
- Stack Overflow discussions
- GitHub examples and templates

---

### Slide 32: Q&A and Thank You

## Questions & Discussion

### Common Questions:
1. **When to use Structured Streaming vs batch processing?**
2. **How to handle schema evolution in production?**
3. **What are the performance implications of stateful operations?**
4. **How to troubleshoot common streaming issues?**
5. **Best practices for monitoring streaming applications?**

### Contact Information:
- **Spark Documentation**: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html
- **Community Forums**: https://stackoverflow.com/questions/tagged/apache-spark-sql
- **GitHub Examples**: https://github.com/apache/spark/tree/master/examples/src/main/python/sql/streaming

---

## Thank You!
### Master Apache Spark Structured Streaming
**Build robust, scalable real-time data processing applications**

---

*This presentation covers Spark Structured Streaming concepts from basic to advanced levels, including practical Python examples, visual explanations, and production best practices. Each slide builds upon previous concepts to ensure comprehensive understanding of stream processing with Apache Spark.*